In [168]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import sys
myclient = MongoClient("mongodb://localhost:27017/")  
try: 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

Connected successfully!!!


In [169]:
db = myclient["Final"]

In [174]:
#Dictionary to get corresponding country collections
videos = {"CA_category_id" : "CAvideos" , "DE_category_id" :"DEvideos","FR_category_id":"FRvideos","GB_category_id":"GBvideos","IN_category_id":"INvideos","JP_category_id":"JPvideos","KR_category_id":"KRvideos","MX_category_id":"MXvideos","RU_category_id":"RUvideos","US_category_id":"USvideos"}
category = {"CAvideos" : "CA_category_id"}

def cattrigger():
    print("1. Display all videos or top 10 channels from selected category")
    print("2. Display the count of videos in a particular category ")
    print("3. Display the videos in a category from a particular trending date")
    print("4. Return to main menu")
    inp = input('Enter option: ')
    if inp == '4':
        return
    retval = getcountry()
    lst = getlst(retval)
    inp2 = getcatval(lst)
    if inp == '1':
        df = fetch_bycat(inp2,retval)
    elif inp == '2':
        df = fetch_bycatcount(inp2,retval)
    elif inp == '3':
        fetch_bycatdate(inp2,retval)
    elif inp < '1' or inp > '4':
        print("Wrong input.Try again...")
        cattrigger()
        #return df
        
def getcountry(): 
    while True: 
        print("1. Category for Canada")
        print("2. Category for Germany")
        print("3. Category for France")
        print("4. Category for Great Britain")
        print("5. Category for India")
        print("6. Category for Japan")
        print("7. Category for South Korea")
        print("8. Category for Mexico")
        print("9. Category for Russia")
        print("10. Category for United States")
        inp = input('Enter your option: ')

        if inp == '1':
            return "CA_category_id"
        elif inp == '2':
            return "DE_category_id"
        elif inp == '3':
            return "FR_category_id"
        elif inp =='4':
            return "GB_category_id"
        elif inp == '5':
            return "IN_category_id"
        elif inp == '6':
            return "JP_category_id"
        elif inp == '7':
            return "KR_Category_id"
        elif inp == '8':
            return "MX_category_id"
        elif inp == '9':
            return "RU_category_id"
        elif inp == '10':
            return "US_category_id"
        elif inp < '1' or inp > '10':
            print("Wrong input, Try again...")
            getcountry()
        
            
    
    

def getlst(cat):
    lst = []
    Collection = db[cat] 
    get_categoryno = Collection.find({})
    for val in get_categoryno:
        lst.append(val['items__snippet__title'])
    return lst
def getcatval(lst):
    for i in range(len(lst)):
        print(str(i+1) +" " + lst[i])
    inp = input("Enter the category number")
    return inp
#important query gets unique top 10 channels from category
def fetch_bycat(inp,collec):
    Collection = db[str(videos[collec])]
    print("1. Display all videos from selected category")
    print("2. Display top 10 channels from selected category")
    inpp = input('Enter the choice: ')
    if inpp == '1':
        get_results = Collection.find({'category_id': inp})
        df =  pd.DataFrame(list(get_results))
        print(df)
    if inpp == '2':
        get_results = Collection.aggregate([{ "$match" : { "category_id": inp} },{"$group": {"_id": "$channel_title", "tmp": {"$first": "$_id"}}},{"$limit": 10}])         
        print()
        print('Top 10 unique channels in the category: ')
        for i in get_results:
            print(i['_id'])
    return

def fetch_bycatcount(inp,collec):
    Collection = db[str(videos[collec])]
    #vall = eval(str(videos[collec]))
    get_results = Collection.aggregate([
        { "$match" : { "category_id": inp} },
        {"$group" : {"_id" : "$category_id", "Num_ofvideos" : {"$sum" : 1}}}])
    for i in get_results:
        if i == None:
            print('No records!')
            break
        else:
            print('Number of videos in the inputed category: ' + str(i['Num_ofvideos']))
    return

def fetch_bycatdate(inp,collec):
    Collection = db[str(videos[collec])]
    print("Enter any one Year:")
    print("17")
    inp2 = input("18 : ")
    while int(inp2)<17 or int(inp2)>18:
        inp2 = input('Enter correct Year!')
    inp3 = input('Select any Month')
    while int(inp3)<0 or int(inp3)>12:
        inp3 = input('Enter correct Month!')
    inp4 = input('Select Day:')
    while int(inp4)<0 or int(inp4)>31:
        inp4 = input('Enter correct Day!: ')
    fin = inp2 + "."  + inp4 +"." + inp3 
    #fin = "17.14.11"
    #inp = '2'
    get_results = Collection.find({ "$and" : [{'category_id': inp},
         {'trending_date' : {'$regex': fin}}]})
    df =  pd.DataFrame(list(get_results))
    if df.empty:
        print("No records found!")
    else:
        print(df)

In [175]:
def vidtrigger():
    while True:
        print("1. Search video")
        print("2. Search channel name")
        print("3. Display video count in Channel")
        print("4. Display by uploaded date")
        print("5. Display video by trending date")
        print("6. Display count of trending videos on a particular date")
        print("7. Display videos by tag and trending date")
        print("8. Display videos with most likes, dislikes or most commented")
        print("9. Display videos with two criterias together")
        print("10. Display all Channels with their video count")
        print("11. Return to Main")
        inp = input("Enter option: ")
        if inp == '11':
            return
        retval = getcountry()
        if inp == '1':
            extractmongo(retval)
        elif inp == '2':
            extractch(retval)
        elif inp == '3':
            fetch_bychacount(retval)
        elif inp == '4':
            fetch_bydate(retval)
        elif inp == '5':
            fetch_bytrend(retval)
        elif inp == '6':
            fetch_bytrendcount(retval)
        elif inp == '7':
            fetch_bytagdate(retval)
        elif inp == '8':
            fetch_topall(retval)
        elif inp == '9':
            fetch_multopall(retval)
        elif inp == '10':
            fetch_allchacount(retval)
        elif inp < '1' or inp > '11':
            print("Wrong input.Try again...")
            vidtrigger()

#inp = input('Enter the channel name')

    
def extractmongo(collec):
    Collection = db[str(videos[collec])] 
    inp = input("Enter the String to be searched")
    if len(inp)<5:
        print('String must have atleast 5 letters')
        while len(inp)<5:
            inp = input("Enter the String Again")
    get_extracted = Collection.find({'title' : {'$regex': inp}})
    df =  pd.DataFrame(list(get_extracted))
    #put constraint of atleast 5 words
    #df = df[df['title'].str.contains(inp)]
    print(df)
    if len(df)<0:
        print("No Matching Results!")
        return 0
    
def extractch(collec):
    Collection = db[str(videos[collec])] 
    inp = input("Enter the String to be searched")
    if len(inp)<5:
        print('String must have atleast 5 letters')
        while len(inp)<5:
            inp = input("Enter the String Again")
    #put constraint of atleast 5 words
    get_extracted = Collection.find({'channel_title' : {'$regex': inp}})
    df =  pd.DataFrame(list(get_extracted))
    print(df)
    if len(df)<0:
        print("No Matching Results!")
        return 0
    
def fetch_bychacount(collec):
    Collection = db[str(videos[collec])]
    inp = input('Enter the String to be searched: ')
    get_results = Collection.aggregate([
        { "$match" : { "channel_title": inp} },
        {"$group" : {"_id" : "$channel_title", "Num_ofvideos" : {"$sum" : 1}}}])
    for i in get_results:
        if i == None:
            print('No records!')
        else:
            print(str(i['_id']) + " has " + str(i['Num_ofvideos'])  +" videos")
        
def fetch_allchacount(collec):
    Collection = db[videos[collec]]
    get_results = Collection.aggregate([
        {"$group" : {"_id" : "$channel_title", "Num_ofvideos" : {"$sum" : 1}}}])
    for i in get_results:
        if i == None:
            print('No records!')
            break
        else:
            print(str(i['_id']) + " has " + str(i['Num_ofvideos'])  +" videos")

def fetch_bydate(collec):
    Collection = db[str(videos[collec])]
    print("Enter any one Year:")
    print("2017")
    inp = input("2018 : ")
    while int(inp)<2017 or int(inp)>2018:
        inp = input('Enter correct Year!')
    inp2 = input('Select any Month: ')
    while int(inp2)<0 or int(inp2)>12:
        inp2 = input('Enter correct Month!')
    inp3 = input('Select Day: ')
    while int(inp3)<0 or int(inp3)>31:
        inp3 = input('Enter correct Day!')
    fin = inp + "-"  + inp2 +"-" + inp3 
    #fin = "2017-11-12"
    get_extracted = Collection.find({'publish_time' : {'$regex': fin}})
    df =  pd.DataFrame(list(get_extracted))
    if df.empty:
        print("No records found!")
    else:
        print(df)
    
def fetch_bytrend(collec):
    Collection = db[str(videos[collec])]
    print("1. Trending videos on a a particular date")
    print("2. Top liked video in country on a particular date")
    print("3. Top viewed video in country on a particular date")
    inpp = input('Enter the choice: ')
    print("Enter any one Year:")
    print("17")
    inp = input("18 : ")
    while int(inp)<17 or int(inp)>18:
        inp = input('Enter correct Year!')
    inp2 = input('Select any Month: ')
    while int(inp2)<0 or int(inp2)>12:
        inp2 = input('Enter correct Month!')
    inp3 = input('Select Day: ')
    while int(inp3)<0 or int(inp3)>31:
        inp3 = input('Enter correct Day!')
    fin = inp + "."  + inp3 +"." + inp2 
    #fin = "17.14.11"
    if inpp == '1':
        get_extracted = Collection.find({'trending_date' : {'$regex': fin}})
    elif inpp == '2':
        get_extracted = Collection.find({'trending_date' : {'$regex': fin}}).sort([('likes', -1)]).limit(1)
    elif inpp == '3':
        get_extracted = Collection.find({'trending_date' : {'$regex': fin}}).sort([('views', -1)]).limit(1)
    elif inp < '1' or inp > '3':
            print("Wrong input. Try again..")
            fetch_bytrend(collec)
    df =  pd.DataFrame(list(get_extracted))
    if df.empty:
        print("No records found!")
    else:
        print(df['title'])
        print("likes " + df['likes'] +" dislikes "+ df['dislikes'] +" comment count "+ df['comment_count'] )
        print(" views: " +df['views'])
        
def fetch_bytrendcount(collec):
    Collection = db[str(videos[collec])]
    print("Enter any one Year:")
    print("17")
    inp = input("18 : ")
    while int(inp)<17 or int(inp)>18:
        inp = input('Enter correct Year!')
    inp2 = input('Select any Month: ')
    while int(inp2)<0 or int(inp2)>12:
        inp2 = input('Enter correct Month!')
    inp3 = input('Select Day: ')
    while int(inp3)<0 or int(inp3)>31:
        inp3 = input('Enter correct Day!')
    fin = inp + "."  + inp3 +"." + inp2 
    #fin = "17.14.11"
    get_results = Collection.aggregate([
        { "$match" : {'trending_date' : {'$regex': fin}} },
        {"$group" : {"_id" : "$trending_date", "Num_ofvideos" : {"$sum" : 1}}}])
    for i in get_results:
        print(str(i['Num_ofvideos']) + " videos were trending on " + str(i['_id']))

#as per trend date
def fetch_bytagdate(collec):
    Collection = db[str(videos[collec])]
    print("1. Display all videos from inputed tag")
    print("2. Display top 10 channels from inputed tag")
    inpp = input('Enter the choice: ')
    inp = input('Enter the tag to be searched!')
    if inpp == '1':
        print("Enter any one Year: ")
        print("17")
        inp2 = input("18 : ")
        while int(inp2)<17 or int(inp2)>18:
            inp2 = input('Enter correct Year!')
        inp3 = input('Select any Month: ')
        while int(inp3)<0 or int(inp3)>12:
            inp3 = input('Enter correct Month!')
        inp4 = input('Select Day: ')
        while int(inp4)<0 or int(inp4)>31:
            inp4 = input('Enter correct Day! ')
        fin = inp2 + "."  + inp4 +"." + inp3 
        get_results = Collection.find({ "$and" : [{'tags': {'$regex': inp}},
             {'trending_date' : {'$regex': fin}}]})
        df =  pd.DataFrame(list(get_results))
        if df.empty:
            print("No records found!")
        else:
            print(df)
    elif inpp == '2':
        get_results = Collection.aggregate([{ "$match" : { 'tags': {'$regex': inp} }},{"$group": {"_id": "$channel_title", "tmp": {"$first": "$_id"}}},{"$limit": 10}]) 
        print('Top 10 unique channels by tags: ')
        for i in get_results:
            print(i['_id'])
    elif inp < '1' or inp > '2':
            print("Wrong input. Try again..")
            fetch_bytagdate(collec)
        
#likes by country,dislikes, most commented on
def fetch_topall(collec):
    Collection = db[str(videos[collec])]
    while True:
        print("1. Top liked video in country")
        print("2. Top disliked video in country")
        print("3. Top commented video in country")
        print("4. Top viewed video in country")
        inp = input("Enter choice! : ")
        if inp == '1':
            get_results = Collection.find().sort([('likes', -1)]).limit(1)
        if inp == '2':
            get_results = Collection.find().sort([('dislikes', -1)]).limit(1)
        if inp == '3':
            get_results = Collection.find().sort([('comment_count', -1)]).limit(1)
        if inp == '4':
            get_results = Collection.find().sort([('views', -1)]).limit(1)
        elif inp < '1' or inp > '4':
            print("Wrong input. Try again..")
            fetch_topall(collec)
        df =  pd.DataFrame(list(get_results))
        if df.empty:
            print("No records found!")
        else:
            print(df['title'] +" " )
            print("likes " + df['likes'] +" dislikes "+ df['dislikes'] +" comment-count "+ df['comment_count'])
            print(" views: " +df['views'])
        return

def fetch_multopall(collec):
    Collection = db[str(videos[collec])]
    while True:
        print("1. Top liked & most commented video in country")
        print("2. Top disliked & most commented video in country") 
        print("3. Video with most likes,views & comments")
        print("4. Video with most dislikes,views & comments")
        inp = input("Enter choice! : ")
        if inp == '1':
            get_results = Collection.find().sort([('likes',-1),('comment_count', -1)]).limit(1)
        if inp == '2':
            get_results = Collection.find().sort([('dislikes',-1),('comment_count', -1)]).limit(1)
        if inp == '3':
            get_results = Collection.find().sort([('likes',-1),('comment_count', -1),('views',-1)]).limit(1)
        if inp == '4':
            get_results = Collection.find().sort([('dislikes',-1),('comment_count', -1),('views',-1)]).limit(1)
        elif inp < '1' or inp > '4':
            print("Wrong input.Try again...")
            fetch_multopall(collec)
        df =  pd.DataFrame(list(get_results))
        if df.empty:
            print("No records found!")
        else:
            print(df['title'] +" " )
            print("likes " + df['likes'] +" dislikes "+ df['dislikes'] +" comment-count "+ df['comment_count'])
            print(" views: " +df['views'])
        return
        
#def resultss(inp,collect):
#     Collection = db[str(videos[collec])] 
#     results = Collection.find({},{'video_id':inp})
#     for res in results:
#         print(res)

In [172]:
#make for category to retrieve all data
def complextrig():
    print('Note- The below queries are all one step ')
    print('1. Displays 10 unique videos from a category')
    print('2. Displays 10 unique videos from a category for 2 countries together')
    print('3. Displays 10 Channels from a category inputted')
    print("4. Return to Main")
    inp = input("Enter option: ")
    if inp == '4':
        return
    retval = getcountry()
    if inp == '1':
        getfrommultiple(retval)
    elif inp == '2':
        print('Select the second country choice')
        retval2 = getcountry()
        while True:
            if retval == retval2:
                print('Please select second country: ')
                retvla2 = get_country()
            else:
                break
        fetch_multicountry(retval,retval2)
    elif inp == '3':
        getchabycategory(retval)
    elif inp < '1' or inp > '4':
            print("Wrong input.Try again...")
            complextrig()
    
def getfrommultiple(collec):
    Collection = db[collec]
    inp = input('Enter the category of videos you want to display! : ')
    get_results = Collection.aggregate([
    {
        "$lookup": {
            'from': videos[collec],
            'localField': "category_id",
            'foreignField': "items__id",
            'as': "categorydata"
        }
    },
    {'$match' :{'items__snippet__title' : {'$regex' : inp } }
                     },
    { '$unwind': { 'path': "$categorydata"}},
                    {'$project' :
                        {'categorydata.title': 1 ,'categorydata.views':1}
                     },{ '$limit' : 10}
    ])
    for i in get_results:
        pprint(i['categorydata']['title'] + " with views: " +i['categorydata']['views'])

#make for category to retrieve all data
def fetch_multicountry(collec,collec2):
    inp = input('Enter the Category String! : ')
    Collection = db[collec]
    Collection2 = db[collec2]
    get_results = Collection.aggregate([
        {
            "$lookup": {
                'from': videos[collec],
                'localField': "category_id",
                'foreignField': "items__id",
                'as': "categorydata"
            }
        },
        {'$match' :{'items__snippet__title' : {'$regex' : inp } }
                         },
        { '$unwind': { 'path': "$categorydata"}},
                     {'$project' :
                        {'categorydata.title': 1 ,'categorydata.views':1}
                     },{ '$limit' : 10}
        
    ])
    get_results2 = Collection2.aggregate([
        {
            "$lookup": {
                'from': videos[collec2],
                'localField': "category_id",
                'foreignField': "items__id",
                'as': "categorydata"
            }
        },
        {'$match' :{'items__snippet__title' : {'$regex' : inp } }
                         },
        { '$unwind': { 'path': "$categorydata"}},
                     {'$project' :
                        {'categorydata.title': 1 ,'categorydata.views':1}
                     },{ '$limit' : 10}
    ])
    print('Videos from first country selected, for category : ' +inp )
    for i in get_results:
        pprint(i['categorydata']['title'] + " with views: " +i['categorydata']['views'])
    print('____________________________________________________________________________')
    print('Videos from second country selected, for category : ' +inp)
    for i in get_results2:
        pprint(i['categorydata']['title'] + " with views: " +i['categorydata']['views'])
    
def getchabycategory(collec):
    Collection = db[collec]
    inp = input('Enter the category of videos you want to display! : ')
    get_results = Collection.aggregate([
    {
        "$lookup": {
            'from': videos[collec],
            'localField': "category_id",
            'foreignField': "items__id",
            'as': "categorydata"
        }
    },
    {'$match' :{'items__snippet__title' : {'$regex' : inp } }
                     },
    { '$unwind': { 'path': "$categorydata"}},
                    {'$project' :
                        {'categorydata.channel_title': 1}
                     },{ '$limit' : 10}
    ])
    for i in get_results:
        pprint(i['categorydata']['channel_title'])

In [149]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
def main():
    while True:
        print('\n')
        print("1. Filter with category")
        print("2. Filter with Video Stats")
        print("3. One step complex")
        print("4. To exit!")
        inp = input("Enter option: ")
        if inp == '1':
            df = cattrigger()
        elif inp == '2':
            df = vidtrigger()
        elif inp == '3':
            complextrig()
        elif inp == '4':
            raise StopExecution
        elif inp < '1' or inp > '4':
            print("Wrong input.Try again...")
            main()
    return

In [173]:
main()



1. Filter with category
2. Filter with Video Stats
3. One step complex
4. To exit!
Enter option: 4
